### LeNet5
https://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.tanh = nn.Tanh()
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1)
        self.l1 = nn.Linear(120, 84)
        self.l2 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = x # N x 1 x 28 x 28
        x = self.tanh(self.conv1(x)) # N x 6 x 28 x 28
        x = self.avgpool(x) # N x 6 x 14 x 14
        x = self.tanh(self.conv2(x)) # N x 16 x 10 x 10
        x = self.avgpool(x) # N x 16 x 5 x 5
        x = self.tanh(self.conv3(x)) # N x 120 x 1 x 1
        x = x.reshape(x.shape[0], -1)
        x = self.tanh(self.l1(x))
        x = self.l2(x)
        return x

In [ ]:
def Dataset(batch_size):
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0,), (1,))
    ])
    trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                            download=False, transform=transform)
    trainset, valset = torch.utils.data.random_split(trainset, [57500, 2500])
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                            shuffle=True)
    testset = torchvision.datasets.MNIST(root='./data', train=False,
                                        download=False, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False)
    return trainloader, valloader, testloader

In [ ]:
model = LeNet()

In [ ]:
lr = 0.01
epochs = 20
batch = 64
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data, target in loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    model.train()
    return 100 * correct / total

def train_model(model, trainloader, valloader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (data, target) in enumerate(trainloader, 0):
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        epoch_loss = running_loss / len(trainloader)
        train_accuracy = calculate_accuracy(trainloader, model)
        val_accuracy = calculate_accuracy(valloader, model)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, "
              f"Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")
    print('Training completed.')

def test_model(model, testloader):
    test_accuracy = calculate_accuracy(testloader, model)
    print(f'Test Accuracy: {test_accuracy:.2f}%')

In [3]:
trainloader, valloader, testloader = Dataset(batch)

train_model(model, trainloader, valloader, criterion, optimizer, epochs)
test_model(model, testloader)


Epoch [1/20], Loss: 1.4156, Train Accuracy: 86.10%, Validation Accuracy: 87.52%
Epoch [2/20], Loss: 0.3862, Train Accuracy: 90.95%, Validation Accuracy: 91.44%
Epoch [3/20], Loss: 0.2675, Train Accuracy: 93.23%, Validation Accuracy: 93.80%
Epoch [4/20], Loss: 0.2018, Train Accuracy: 94.79%, Validation Accuracy: 94.80%
Epoch [5/20], Loss: 0.1592, Train Accuracy: 95.85%, Validation Accuracy: 95.60%
Epoch [6/20], Loss: 0.1311, Train Accuracy: 96.55%, Validation Accuracy: 96.60%
Epoch [7/20], Loss: 0.1114, Train Accuracy: 96.97%, Validation Accuracy: 97.04%
Epoch [8/20], Loss: 0.0975, Train Accuracy: 97.33%, Validation Accuracy: 97.36%
Epoch [9/20], Loss: 0.0869, Train Accuracy: 97.58%, Validation Accuracy: 97.56%
Epoch [10/20], Loss: 0.0785, Train Accuracy: 97.78%, Validation Accuracy: 97.92%
Epoch [11/20], Loss: 0.0721, Train Accuracy: 97.96%, Validation Accuracy: 97.84%
Epoch [12/20], Loss: 0.0667, Train Accuracy: 98.07%, Validation Accuracy: 97.92%
Epoch [13/20], Loss: 0.0622, Train A

In [ ]:
torch.save(model.state_dict(), 'LeNet.pth')